In [1]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *
from pyspark import StorageLevel

import numpy as np
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("mode.chained_assignment", None)

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [3]:
# !pip install scikit-plot

In [4]:
import sklearn
import scikitplot as skplt
from sklearn.metrics import classification_report, confusion_matrix, precision_score

<hr />
<hr />
<hr />

In [5]:
result_schema = StructType([
                    StructField('experiment_filter', StringType(), True),
                    StructField('undersampling_method', StringType(), True),
                    StructField('undersampling_column', StringType(), True),
                    StructField('filename', StringType(), True),
                    StructField('experiment_id', StringType(), True),
                    StructField('n_covid', IntegerType(), True),
                    StructField('n_not_covid', IntegerType(), True),
                    StructField('model_name', StringType(), True),
                    StructField('model_seed', StringType(), True),
                    StructField('model_maxIter', IntegerType(), True),
                    StructField('model_maxDepth', IntegerType(), True),
                    StructField('model_maxBins', IntegerType(), True),
                    StructField('model_minInstancesPerNode', IntegerType(), True),
                    StructField('model_minInfoGain', FloatType(), True),
                    StructField('model_featureSubsetStrategy', StringType(), True),
                    StructField('model_n_estimators', IntegerType(), True),
                    StructField('model_learning_rate', FloatType(), True),
                    StructField('model_impurity', StringType(), True),
                    StructField('model_AUC_ROC', StringType(), True),
                    StructField('model_AUC_PR', StringType(), True),
                    StructField('model_covid_precision', StringType(), True),
                    StructField('model_covid_recall', StringType(), True),
                    StructField('model_covid_f1', StringType(), True),
                    StructField('model_not_covid_precision', StringType(), True),
                    StructField('model_not_covid_recall', StringType(), True),
                    StructField('model_not_covid_f1', StringType(), True),
                    StructField('model_avg_precision', StringType(), True),
                    StructField('model_avg_recall', StringType(), True),
                    StructField('model_avg_f1', StringType(), True),
                    StructField('model_avg_acc', StringType(), True),
                    StructField('model_TP', StringType(), True),
                    StructField('model_TN', StringType(), True),
                    StructField('model_FN', StringType(), True),
                    StructField('model_FP', StringType(), True),
                    StructField('model_time_exec', StringType(), True),
                    StructField('model_col_set', StringType(), True)
                          ])

In [6]:
# GBT PARAMS

# params_dict = {'maxIter': 20, ===> [100, 200, 300]
#               'maxDepth': 3, ===> [3, 5, 10]
#               'maxBins': 20, ===> [16, 32, 64]
#               'learningRate': 0.01,===> [0.01, 0.1, 0.5]
#               'loss': 'logLoss'}  ===> ["logLoss", "leastSquaresError", "leastAbsoluteError"]

# .addGrid(gbt.seed, [2021]) \
#                             .addGrid(gbt.maxIter, [100, 300, 500]) \
#                             .addGrid(gbt.maxDepth, [3, 5, 10, 15]) \
#                             .addGrid(gbt.maxBins, [50, 200]) \
#                             .addGrid(gbt.minInstancesPerNode, [1, 15]) \
#                             .addGrid(gbt.minInfoGain, [0, 3]) \
#                             .addGrid(gbt.featureSubsetStrategy, ['all', 'auto']).build()


# XGBOOST PARAMS
#     'max_depth': range (2, 10, 1),
#     'n_estimators': range(60, 220, 40),
#     'learning_rate': [0.1, 0.01, 0.05]


# RF PARAMS
#                             .addGrid(rf.seed, [2021]) \
#                             .addGrid(rf.numTrees, range(10, 150, 30)) \
#                             .addGrid(rf.maxDepth, range(3, 15, 2)) \
#                             .addGrid(rf.maxBins, range(10, 90, 30)) \
#                             .addGrid(rf.minInstancesPerNode, range(1, 100, 20)) \
#                             .addGrid(rf.minInfoGain, range(0, 10, 2)) \
#                             .addGrid(rf.featureSubsetStrategy, ['all', 'auto', 'onethird', 'sqrt', 'log2']) \
#                             .addGrid(rf.impurity, ['gini', 'entropy']).build()

<hr />
<hr />
<hr />

In [7]:
cols_sets = {'cols_set_1': ['NU_NOTIFIC', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO', 
                            'AGE_GROUP', 'DIST_PRI_NOTIFIC_Q',
                            'DIST_PRI_INTERNA_Q', 'DIST_PRI_ENTUTI_Q', 'DIST_PRI_SAIDUTI_Q', 'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_ENCERRA_Q',
                            'SYMP_GROUP1', 'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
                            'RF_GROUP1', 'RF_GROUP2', 'RF_GROUP3', 'RF_GROUP4',
                            'SUPORT_VEN', 'UTI', 'HOSPITAL',
                            'DIST_PRI_RAIOX_Q', 'DIST_PRI_COLETA_Q', 'DIST_PRI_TOMO_Q', 'DIST_PRI_IF_Q', 'DIST_PRI_TRA_Q', 'DIST_PRI_PCR_Q', 'DIST_PRI_SOR_Q',
                            'GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q', 'GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q', 'GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q', 'GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q',
                            'GMR_TRANSIT_STATIONS_2WEEKS_Q', 'GMR_RETAIL_AND_RECREATION_2WEEKS_Q', 'GMR_RESIDENTIAL_PERCENT_2WEEKS_Q', 'GMR_WORKPLACES_PERCENT_2WEEKS_Q',
                            'INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q', 'INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q'],
             'cols_set_2': ['CS_SEXO','CS_RACA', 'CRITERIO', 'SURTO_SG', 'SUPORT_VEN', 'EVOLUCAO',
                            'OUTRO_SIN', 'AGE_GROUP', 
                            'SYMP_GROUP1',  'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
                            'RF_GROUP1', 'RF_GROUP2', 'RF_GROUP3',  'RF_GROUP4',
                            'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_ENCERRA_Q', 'DIST_PRI_INTERNA_Q',
                            'DIST_PRI_NOTIFIC_Q', 'DIST_PRI_COLETA_Q', 'DIST_PRI_PCR_Q', 'CLASSI_FIN'],
             'cols_set_3': ['SYMP_GROUP1', 'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
                            'AGE_GROUP', 'CS_SEXO', 'CS_RACA', 'EVOLUCAO', 'CS_ESCOL_N',
                            'UTI', 'SUPORT_VEN', 'DIST_PRI_ENTUTI_Q', 'DIST_PRI_INTERNA_Q',
                            'CLASSI_FIN']}

<hr />
<hr />
<hr />

In [8]:
def run_rf(exp_df, params_dict, cols, filename, exp_name, experiment_filter, 
            undersampling_method, undersampling_column, experiment_id):
    import time
    start_time = time.time()
    
    n_covid = exp_df.filter(F.col('CLASSI_FIN') == 1.0).count()
    n_not_covid = exp_df.filter(F.col('CLASSI_FIN') == 0.0).count()
    
    
    id_cols = ['NU_NOTIFIC', 'CLASSI_FIN']

    labelIndexer = StringIndexer(inputCol="CLASSI_FIN", outputCol="indexedLabel").fit(exp_df)    
    
    input_cols = [x for x in exp_df.columns if x not in id_cols]
    assembler = VectorAssembler(inputCols = input_cols, outputCol= 'features')
    exp_df = assembler.transform(exp_df)
    
    # Automatically identify categorical features, and index them.
    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=20).fit(exp_df)
    
    # Split the data into training and test sets (30% held out for testing)
    (trainingData, testData) = exp_df.randomSplit([0.7, 0.3])
    trainingData = trainingData.persist(StorageLevel.MEMORY_ONLY)
    testData = testData.persist(StorageLevel.MEMORY_ONLY)
    
    # Train a RandomForest model.
    rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                               numTrees = params_dict['numTrees'], maxDepth = params_dict['maxDepth'], maxBins = params_dict['maxBins'])
    
    # Convert indexed labels back to original labels.
    labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

    # Chain indexers and forest in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])
    
    # Train model.  This also runs the indexers.
    model = pipeline.fit(trainingData)
    
    # Make predictions.
    predictions = model.transform(testData)    
    
    
    pred = predictions.select(['CLASSI_FIN', 'predictedLabel'])\
                  .withColumn('predictedLabel', F.col('predictedLabel').cast('double'))\
                  .withColumn('predictedLabel', F.when(F.col('predictedLabel') == 1.0, 'covid').otherwise('n-covid'))\
                  .withColumn('CLASSI_FIN', F.when(F.col('CLASSI_FIN') == 1.0, 'covid').otherwise('n-covid'))\
                  .toPandas()

    y_true = pred['CLASSI_FIN'].tolist()
    y_pred = pred['predictedLabel'].tolist()
    
    report = classification_report(y_true, y_pred, output_dict=True)
    
    
    evaluator_ROC = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
    accuracy_ROC = evaluator_ROC.evaluate(predictions)


    
    evaluator_PR = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction", metricName="areaUnderPR")
    accuracy_PR = evaluator_PR.evaluate(predictions)
    
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    result_dict = {}
    
    result_dict['experiment_filter'] = experiment_filter
    result_dict['undersampling_method'] = undersampling_method
    result_dict['undersampling_column'] = undersampling_column
    result_dict['filename'] = filename
    result_dict['experiment_id'] = experiment_id
    result_dict['n_covid'] = n_covid
    result_dict['n_not_covid'] = n_not_covid
    result_dict['model_name'] = 'RF'
    result_dict['model_maxIter'] = params_dict['numTrees']
    result_dict['model_maxDepth'] = params_dict['maxDepth']
    result_dict['model_maxBins'] = params_dict['maxBins']
    result_dict['model_AUC_ROC'] = accuracy_ROC
    result_dict['model_AUC_PR'] = accuracy_PR
    result_dict['model_covid_precision'] = report['covid']['precision']
    result_dict['model_covid_recall'] = report['covid']['recall']
    result_dict['model_covid_f1'] = report['covid']['f1-score']
    result_dict['model_not_covid_precision'] = report['n-covid']['precision']
    result_dict['model_not_covid_recall'] = report['n-covid']['recall']
    result_dict['model_not_covid_f1'] = report['n-covid']['f1-score']
    result_dict['model_avg_precision'] = report['macro avg']['precision']
    result_dict['model_avg_recall'] = report['macro avg']['recall']
    result_dict['model_avg_f1'] = report['macro avg']['f1-score']
    result_dict['model_avg_acc'] = report['accuracy']
    result_dict['model_TP'] = conf_matrix[0][0]
    result_dict['model_TN'] = conf_matrix[1][1]
    result_dict['model_FN'] = conf_matrix[0][1]
    result_dict['model_FP'] = conf_matrix[1][0]
    result_dict['model_time_exec'] = time.time() - start_time
    result_dict['model_col_set'] = cols
    
    return result_dict

<hr />
<hr />
<hr />

# Running RF on 10 samples for each experiment
### 3x col sets -> ['cols_set_1', 'cols_set_2', 'cols_set_3']
### 3x model_numTrees -> [200, 500, 700]
### 3x model_maxDepth -> [5, 10, 15]
### 3x model_maxBins -> [16, 32, 64]
Total: 10 * 3 * 3 * 3 * 3 = 810

In [9]:
# lists of params
model_numTrees = [100] # [200, 500, 700]
model_maxDepth = [7] # [5, 10, 15]
model_maxBins = [32] # [16, 32, 64]

In [10]:
experiments = []

### Datasets: strat_samp_lab_agegrp

In [11]:
for id_exp in range(3):
    for col_set in list(cols_sets.keys()):
        str_file = 'gs://ai-covid19-datalake/trusted/experiment_map/datasets/rose_nofilter_nostratsamp_' + col_set + '_' + str(id_exp)
        exp_dataframe = spark.read.parquet(str_file)
        print('read it')
        filename = str_file.split('/')[-1].split('.')[0] # strat_samp_lab_agegrp_0
        experiment_name = filename
        undersampling_method = "ROSE"
        experiment_filter = ""
        undersampling_column = ""
        
        for numTrees in model_numTrees:
            for maxDepth in model_maxDepth:
                for maxBins in model_maxBins:
                    params_dict = {'numTrees': numTrees, 'maxDepth': maxDepth, 'maxBins': maxBins}
                    try:                     
                        model = run_rf(exp_dataframe, params_dict, col_set, filename, experiment_name, experiment_filter, undersampling_method, undersampling_column, id_exp)
                        experiments.append(model)

                        print(experiment_name, str(params_dict), str(model['model_AUC_PR']), str(model['model_time_exec']))
                    except:
                        print('Something wrong with the exp: {}, {}, {}'.format(filename, params_dict, col_set))

read it
rose_nofilter_nostratsamp_cols_set_1_0 {'numTrees': 100, 'maxDepth': 7, 'maxBins': 32} 0.7928598947997897 36.52442526817322
read it
rose_nofilter_nostratsamp_cols_set_2_0 {'numTrees': 100, 'maxDepth': 7, 'maxBins': 32} 0.76792291763222 27.726691722869873
read it
rose_nofilter_nostratsamp_cols_set_3_0 {'numTrees': 100, 'maxDepth': 7, 'maxBins': 32} 0.7500497352136875 27.99180817604065
read it
rose_nofilter_nostratsamp_cols_set_1_1 {'numTrees': 100, 'maxDepth': 7, 'maxBins': 32} 0.8259015880174463 27.53451132774353
read it
rose_nofilter_nostratsamp_cols_set_2_1 {'numTrees': 100, 'maxDepth': 7, 'maxBins': 32} 0.7768881296007201 29.50003480911255
read it
rose_nofilter_nostratsamp_cols_set_3_1 {'numTrees': 100, 'maxDepth': 7, 'maxBins': 32} 0.7320767512007377 31.547686100006104
read it
rose_nofilter_nostratsamp_cols_set_1_2 {'numTrees': 100, 'maxDepth': 7, 'maxBins': 32} 0.7933927555761886 28.804713487625122
read it
rose_nofilter_nostratsamp_cols_set_2_2 {'numTrees': 100, 'maxDepth'

In [12]:
cols = ['experiment_filter', 'undersampling_method', 'undersampling_column', 'filename', 'experiment_id', 'n_covid', 'n_not_covid', 'model_name', 'model_maxIter', 'model_maxDepth', 'model_maxBins', 'model_AUC_ROC', 'model_AUC_PR', 'model_covid_precision', 'model_covid_recall', 'model_covid_f1', 'model_not_covid_precision', 'model_not_covid_recall', 'model_not_covid_f1', 'model_avg_precision', 'model_avg_recall', 'model_avg_f1', 'model_avg_acc', 'model_TP', 'model_TN', 'model_FN', 'model_FP', 'model_time_exec', 'model_col_set']

In [13]:
for i in range(len(experiments)):
    for d in list(experiments[i].keys()):
        experiments[i][d] = str(experiments[i][d])

In [14]:
intermed_results = spark.createDataFrame(data=experiments).select(cols)
intermed_results.toPandas()

/usr/lib/spark/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


,experiment_filter,undersampling_method,undersampling_column,filename,experiment_id,n_covid,n_not_covid,model_name,model_maxIter,model_maxDepth,model_maxBins,model_AUC_ROC,model_AUC_PR,model_covid_precision,model_covid_recall,model_covid_f1,model_not_covid_precision,model_not_covid_recall,model_not_covid_f1,model_avg_precision,model_avg_recall,model_avg_f1,model_avg_acc,model_TP,model_TN,model_FN,model_FP,model_time_exec,model_col_set
0,,ROSE,,rose_nofilter_nostratsamp_cols_set_1_0,0,3827,3851,RF,100,7,32,0.8497386280127697,0.7928598947997897,0.8091054313099042,0.8996447602131439,0.8519764507989908,0.8941947565543071,0.7998324958123953,0.8443854995579135,0.8516500939321057,0.8497386280127697,0.8481809751784521,0.8482758620689655,1013,955,113,239,36.52442526817322,cols_set_1
1,,ROSE,,rose_nofilter_nostratsamp_cols_set_2_0,0,3827,3851,RF,100,7,32,0.808262183536788,0.76792291763222,0.7878787878787878,0.856418918918919,0.8207203561311209,0.8357487922705314,0.7601054481546573,0.796134376438104,0.8118137900746596,0.8082621835367881,0.8084273662846124,0.8092161929371232,1014,865,170,273,27.726691722869873,cols_set_2
2,,ROSE,,rose_nofilter_nostratsamp_cols_set_3_0,0,3827,3851,RF,100,7,32,0.8031005729848266,0.7500497352136875,0.7694099378881988,0.8587521663778163,0.8116298116298116,0.8435700575815739,0.7474489795918368,0.7926059513074842,0.8064899977348863,0.8031005729848265,0.8021178814686479,0.8025751072961373,991,879,163,297,27.99180817604065,cols_set_3
3,,ROSE,,rose_nofilter_nostratsamp_cols_set_1_1,1,3827,3851,RF,100,7,32,0.8673437689928553,0.8259015880174463,0.84375,0.8976377952755905,0.8698601102161934,0.8929551692589204,0.8370497427101201,0.8640991589198761,0.8683525846294602,0.8673437689928554,0.8669796345680347,0.8670420095279342,1026,976,117,190,27.53451132774353,cols_set_1
4,,ROSE,,rose_nofilter_nostratsamp_cols_set_2_1,1,3827,3851,RF,100,7,32,0.8213292876215268,0.7768881296007201,0.7948511665325825,0.87511071744907,0.833052276559865,0.8565615462868769,0.7675478577939836,0.8096153846153845,0.8257063564097297,0.8213292876215268,0.8213338305876248,0.8221024258760108,988,842,141,255,29.50003480911255,cols_set_2
5,,ROSE,,rose_nofilter_nostratsamp_cols_set_3_1,1,3827,3851,RF,100,7,32,0.7919418930662908,0.7320767512007377,0.7538712306438468,0.8393829401088929,0.794332331472735,0.8325449385052034,0.7445008460236887,0.7860652076820009,0.7932080845745251,0.7919418930662908,0.790198769577368,0.7902802101576182,925,880,177,302,31.547686100006104,cols_set_3
6,,ROSE,,rose_nofilter_nostratsamp_cols_set_1_2,2,3827,3851,RF,100,7,32,0.8469393706366213,0.7933927555761886,0.8060921248142645,0.9163851351351351,0.8577075098814229,0.9020771513353115,0.7774936061381074,0.8351648351648351,0.854084638074788,0.8469393706366213,0.846436172523129,0.8472634705133645,1085,912,99,261,28.804713487625122,cols_set_1
7,,ROSE,,rose_nofilter_nostratsamp_cols_set_2_2,2,3827,3851,RF,100,7,32,0.8119176952830043,0.7570075508748865,0.7771474878444085,0.8600896860986547,0.8165176670923797,0.8507177033492823,0.7637457044673539,0.8048890900860118,0.8139325955968454,0.8119176952830043,0.8107033785891957,0.8108819657744625,959,889,156,275,27.46751356124878,cols_set_2
8,,ROSE,,rose_nofilter_nostratsamp_cols_set_3_2,2,3827,3851,RF,100,7,32,0.7938000037097034,0.7515535910710142,0.7750410509031199,0.8287971905179983,0.8010182435299109,0.815515610217597,0.7588028169014085,0.7861377108983129,0.7952783305603585,0.7938000037097034,0.7935779772141118,0.7938461538461539,944,862,195,274,27.788637161254883,cols_set_3


In [15]:
# intermed_results.write.parquet('gs://ai-covid19-datalake/trusted/int_exp_results_rf_strat_samp_lab_agegrp-0-9.parquet', mode='overwrite')